This will be a very simple, straightforward tutorial to S4 (see the Tutorial in our S4 documentation)

# 0. Imports

In [1]:
# import S4 (S4 must be installed)
import S4

# 1. Create a simulation object

In order to simulate using S4 (or really *do* anything in S4), you first need to create a simulation object:

In [2]:
S = S4.Simulation()
S.create_new()

we now have an S4 Simulation object in `S` with a new simulation instance that we created with `create_new()`.

# 2. Create the simulation geometry

We are simulating a geometric object that is periodic in $x, y$ with a finite number of distinct layers in $z$. Thus, we must define this geometry:

In [3]:
a_x = 0.75
a_y = 0.0

b_x = 0.0
b_y = 0.75
S.set_lattice([[a_x, a_y],
               [b_x, b_y]])

This creates a square lattice with a lattice vector of length $L=0.75$ scaled units.

Let's now define the number of basis functions we will use (*Note: the number required to accurately approximate the optical behavior will be system dependent and will require a sensitivity study*)

In [4]:
# use 100 basis functions
S.set_num_g(100)

# 3. Now define the materials in the simulation

We are simulating a physical object composed of real materials, so we need to add these materials to the simulation (*Note: if you have materials with appreciable dispersion, you must change the dielectric constant if/when you change the frequency*)

In [5]:
S.add_material("Vacuum", [1.0, 0.0])
S.add_material("Silicon", [12.0, 0.01])

We have added a material (technically the lack thereof) `Vacuum` with a dielectric onstant of $1$ and a material `Silicon` with a dielectric constant of $12 + 0.01i$

# 4. Create the layers of the object

We will now create the layers in the object we will simulate. In this example, we have a single slab; however, this slab is technically requires 3 layers to be accurately simulated because we must include the environment above and below the slab:

In [6]:
S.add_layer("top", 0.0, "Vacuum")
S.add_layer("slab", 0.5, "Silicon")
S.add_layer("bottom", 0.0, "Vacuum")

We added a semi-infinite `Vacuum` layer (we set the thickness to $0$ because it is largely irrelevant to this simulation), a middle slab layer made of silicon with a thickness of $0.5$, and a bottom semi=infinite `Vacuum` layer.

# 5. Add patterning to layers

RCWA can be used to simulate homogenous layers, but is effectively overkill. We will actually simulate an object that requires RCWA, so we need to add a pattern to the slab layer:

In [7]:
S.set_layer_pattern_circle("slab",        # Layer to pattern
                           "Vacuum",      # Material inside circle
                           [0.0, 0.0],    # center of pattern
                           0.2)           # radius of circle

This creates a cylinder of radius $r = 0.2$ in the middle layer

# 6. Set the incident light properties

We now need to set the properties of the incident light

In [8]:
S.set_excitation_planewave([0.0, 0.0],    # phi, theta
                           [1.0, 0.0],    # s-polarization (amp, phase)
                           [0.0, 0.0])    # p-polarization (amp, phase)
S.set_frequency(0.4)                      # 1 / wavelength

(I skip the # 7 currently in the documentation)

# 8. Obtain the desired output

Let's run the simulation and get the transmission through the metamaterial

In [11]:
reflection = S.get_poynting_flux("top")

print(reflection)

transmission = S.get_poynting_flux("bottom")

print(transmission)

print(transmission[0] - reflection[1])

[ 1.00000000e+00 -4.52860161e-01 -2.77555756e-17  2.77555756e-17]
[0.54347845 0.         0.         0.        ]
0.9963386107450254


The output of `S.get_poynting_flux` will be:

|   | forward real | backward_real | forward_imaginary | backward_imaginary|
| - | ------------ | ------------- | ----------------- | ----------------- |
| reflection | 1.0   | -0.453           | 0.0               | 0.0               |
| transmission | 0.543   | 0.0           | 0.0               | 0.0               |

(you can ignore `-2.77555756e-17` or similar values)

This means that:

* incident light with a total power of $1$ is input to the system
* light with a power of $0.453$ is reflected (that's what the "-" in $-0.452$ *means*)
* the remainder of the light is transmitted: $0.543 + 0.453 \approx 1$